# Setup & load data

In [ ]:
import pandas as pd
import numpy as np

PATH_TAFSIR = "../data/processed/tafsir_clean.csv"
PATH_QUERIES = "../data/processed/queries_synthetic_all.csv"
OUTPUT_PATH = "../data/processed/relevance_pairs_anchor_random.csv"

# Nama kolom di tafsir_clean.csv
COL_TAFSIR_ID = "tafsir_id"            
COL_TAFSIR_TEXT = "tafsir"             

# Nama kolom di queries_synthetic_all.csv
COL_QUERY_TEXT = "query_text"          
COL_QUERY_TAFSIR_SRC = "tafsir_id_asal"

df_tafsir = pd.read_csv(PATH_TAFSIR)
df_q = pd.read_csv(PATH_QUERIES)

print("tafsir_clean:", df_tafsir.shape)
print("queries_synthetic_all:", df_q.shape)


tafsir_clean: (6236, 7)
queries_synthetic_all: (37557, 14)


# Pastikan tafsir_id_asal dan query_id ada

In [ ]:
# Buat tafsir_id_asal jika belum ada
if COL_QUERY_TAFSIR_SRC not in df_q.columns:
    if not {"surah_x", "ayah_x"}.issubset(df_q.columns):
        raise ValueError("Kolom 'tafsir_id_asal' tidak ada dan tidak ditemukan 'surah' + 'ayah' di df_q.")
    df_q[COL_QUERY_TAFSIR_SRC] = df_q["surah_x"].astype(str) + ":" + df_q["ayah_x"].astype(str)

if "query_id" not in df_q.columns:
    df_q["query_id"] = np.arange(1, len(df_q) + 1)

df_q[[ "query_id", COL_QUERY_TEXT, COL_QUERY_TAFSIR_SRC ]].head()


,query_id,query_text,tafsir_id_asal
0,1,Apa makna dari basmalah dalam memulai bacaan A...,Al-Fātiḥah :1
1,2,Mengapa penting menyebut nama Allah sebelum me...,Al-Fātiḥah :1
2,3,Bagaimana basmalah dapat memberikan keberkahan...,Al-Fātiḥah :1
3,4,Siapa yang dimaksud dengan Yang Maha Pengasih ...,Al-Fātiḥah :1
4,5,Apa perbedaan antara sifat kasih Allah kepada ...,Al-Fātiḥah :1


# Sinkronisasi dengan tafsir_clean (hanya pakai tafsir yang ada)

In [4]:
# Filter hanya query yang tafsir_id_asal-nya memang ada di tafsir_clean
valid_tafsir_ids = set(df_tafsir[COL_TAFSIR_ID].unique())
mask_valid = df_q[COL_QUERY_TAFSIR_SRC].isin(valid_tafsir_ids)

print("Total query:", len(df_q))
print("Query dengan tafsir_id_asal valid:", mask_valid.sum())

df_q = df_q[mask_valid].reset_index(drop=True)


Total query: 37557
Query dengan tafsir_id_asal valid: 37557


# Buat pasangan POSITIF (anchor)

In [5]:
df_pos = df_q[["query_id", COL_QUERY_TEXT, COL_QUERY_TAFSIR_SRC]].copy()
df_pos = df_pos.rename(columns={COL_QUERY_TAFSIR_SRC: COL_TAFSIR_ID})
df_pos["label"] = 1
df_pos["pair_type"] = "pos_anchor"

df_pos.head(), len(df_pos)


(   query_id                                         query_text      tafsir_id  \
 0         1  Apa makna dari basmalah dalam memulai bacaan A...  Al-Fātiḥah :1   
 1         2  Mengapa penting menyebut nama Allah sebelum me...  Al-Fātiḥah :1   
 2         3  Bagaimana basmalah dapat memberikan keberkahan...  Al-Fātiḥah :1   
 3         4  Siapa yang dimaksud dengan Yang Maha Pengasih ...  Al-Fātiḥah :1   
 4         5  Apa perbedaan antara sifat kasih Allah kepada ...  Al-Fātiḥah :1   
 
    label   pair_type  
 0      1  pos_anchor  
 1      1  pos_anchor  
 2      1  pos_anchor  
 3      1  pos_anchor  
 4      1  pos_anchor  ,
 37557)

# Siapkan daftar semua tafsir + parameter negatif per query

In [6]:
all_tafsir_ids = df_tafsir[COL_TAFSIR_ID].tolist()
n_docs = len(all_tafsir_ids)
print("Jumlah dokumen tafsir:", n_docs)

# Berapa negatif random per query?
N_NEG_PER_QUERY = 5  # kamu bisa naikkan nanti jadi 5 misalnya

rng = np.random.default_rng(seed=42)


Jumlah dokumen tafsir: 6236


# Generate NEGATIF acak per query

In [ ]:
neg_rows = []

for _, row in df_q.iterrows():
    qid = row["query_id"]
    qtext = row[COL_QUERY_TEXT]
    tafsir_src = row[COL_QUERY_TAFSIR_SRC]
    
    possible_ids = [tid for tid in all_tafsir_ids if tid != tafsir_src]

    if len(possible_ids) == 0:
        continue  

    size = min(N_NEG_PER_QUERY, len(possible_ids))
    sampled_ids = rng.choice(possible_ids, size=size, replace=False)

    for tid_neg in sampled_ids:
        neg_rows.append({
            "query_id": qid,
            COL_QUERY_TEXT: qtext,
            COL_TAFSIR_ID: tid_neg,
            "label": 0,
            "pair_type": "neg_random",
        })

df_neg = pd.DataFrame(neg_rows)
df_neg.head(), len(df_neg)


(   query_id                                         query_text  \
 0         1  Apa makna dari basmalah dalam memulai bacaan A...   
 1         1  Apa makna dari basmalah dalam memulai bacaan A...   
 2         1  Apa makna dari basmalah dalam memulai bacaan A...   
 3         1  Apa makna dari basmalah dalam memulai bacaan A...   
 4         1  Apa makna dari basmalah dalam memulai bacaan A...   
 
         tafsir_id  label   pair_type  
 0      An-Najm:41      0  neg_random  
 1  Al-Mu'minūn:64      0  neg_random  
 2     Az-Zumar:23      0  neg_random  
 3    An-Nisā' :65      0  neg_random  
 4  Al-Mu'minūn:28      0  neg_random  ,
 187785)

# Gabungkan POS + NEG, buang duplikat

In [8]:
df_pairs = pd.concat([df_pos, df_neg], ignore_index=True)

# Buang duplikat query–tafsir kalau ada
df_pairs = df_pairs.drop_duplicates(subset=["query_id", COL_TAFSIR_ID], keep="first").reset_index(drop=True)

df_pairs.head(), df_pairs["label"].value_counts()


(   query_id                                         query_text      tafsir_id  \
 0         1  Apa makna dari basmalah dalam memulai bacaan A...  Al-Fātiḥah :1   
 1         2  Mengapa penting menyebut nama Allah sebelum me...  Al-Fātiḥah :1   
 2         3  Bagaimana basmalah dapat memberikan keberkahan...  Al-Fātiḥah :1   
 3         4  Siapa yang dimaksud dengan Yang Maha Pengasih ...  Al-Fātiḥah :1   
 4         5  Apa perbedaan antara sifat kasih Allah kepada ...  Al-Fātiḥah :1   
 
    label   pair_type  
 0      1  pos_anchor  
 1      1  pos_anchor  
 2      1  pos_anchor  
 3      1  pos_anchor  
 4      1  pos_anchor  ,
 label
 0    187785
 1     37557
 Name: count, dtype: int64)

# Join dengan teks tafsir (supaya nanti training lebih mudah)

In [9]:
df_pairs = df_pairs.merge(
    df_tafsir[[COL_TAFSIR_ID, COL_TAFSIR_TEXT]],
    on=COL_TAFSIR_ID,
    how="left"
)

df_pairs.rename(columns={COL_TAFSIR_TEXT: "tafsir_text"}, inplace=True)

df_pairs.head()


,query_id,query_text,tafsir_id,label,pair_type,tafsir_text
0,1,Apa makna dari basmalah dalam memulai bacaan A...,Al-Fātiḥah :1,1,pos_anchor,Aku memulai bacaan Al-Qur'an dengan menyebut n...
1,2,Mengapa penting menyebut nama Allah sebelum me...,Al-Fātiḥah :1,1,pos_anchor,Aku memulai bacaan Al-Qur'an dengan menyebut n...
2,3,Bagaimana basmalah dapat memberikan keberkahan...,Al-Fātiḥah :1,1,pos_anchor,Aku memulai bacaan Al-Qur'an dengan menyebut n...
3,4,Siapa yang dimaksud dengan Yang Maha Pengasih ...,Al-Fātiḥah :1,1,pos_anchor,Aku memulai bacaan Al-Qur'an dengan menyebut n...
4,5,Apa perbedaan antara sifat kasih Allah kepada ...,Al-Fātiḥah :1,1,pos_anchor,Aku memulai bacaan Al-Qur'an dengan menyebut n...


# Simpan ke CSV final

In [10]:
df_pairs.to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH, "| rows:", len(df_pairs))


Saved: ../data/processed/relevance_pairs_anchor_random.csv | rows: 225342


# Cek relevence lama dan baru

In [20]:
import pandas as pd

PATH_OLD = "../data/processed/final_relevance_labeled_dataset.csv"
PATH_SYN = "../data/processed/queries_synthetic_all.csv"

df_old = pd.read_csv(PATH_OLD)
df_syn = pd.read_csv(PATH_SYN)

df_old.shape, df_syn.shape


((184655, 3), (37557, 14))

In [21]:
old_queries = (
    df_old["query"]
    .astype(str)
    .str.strip()
    .str.lower()
)

syn_queries = (
    df_syn["query_text"]
    .astype(str)
    .str.strip()
    .str.lower()
)

old_set = set(old_queries.unique())
syn_set = set(syn_queries.unique())

print("Unique query di OLD relevance  :", len(old_set))
print("Unique query di synthetic_all  :", len(syn_set))
print("Query yang ada di KEDUA dataset:", len(old_set & syn_set))
print("Query HANYA di synthetic_all   :", len(syn_set - old_set))
print("Query HANYA di old relevance   :", len(old_set - syn_set))


Unique query di OLD relevance  : 30430
Unique query di synthetic_all  : 35732
Query yang ada di KEDUA dataset: 0
Query HANYA di synthetic_all   : 35732
Query HANYA di old relevance   : 30430


In [22]:
only_in_syn = list(syn_set - old_set)
only_in_old = list(old_set - syn_set)

print("Contoh query HANYA ada di queries_synthetic_all (belum pernah dilabeli):")
for q in only_in_syn[:20]:
    print("-", q)

print("\nContoh query HANYA ada di final_relevance_labeled_dataset (tidak ada di synthetic_all):")
for q in only_in_old[:20]:
    print("-", q)


Contoh query HANYA ada di queries_synthetic_all (belum pernah dilabeli):
- mengapa orang kafir tidak tergerak hatinya untuk beriman padahal melihat fenomena alam?
- mengapa perbuatan seperti mengadu domba dianggap mengotori jiwa?
- apa jawaban orang-orang kafir ketika ditanya tentang kebangkitan?
- apa maksud ayat-ayat allah yang dihamparkan di alam ini?
- bagaimana keyakinan yang keliru mempengaruhi kondisi hati seseorang?
- mengapa allah tidak dianggap zalim meskipun memberikan azab tersebut?
- apa yang dilakukan oleh sekelompok orang sehingga mereka dianggap terlibat dalam pembunuhan?
- siapa yang dimaksud dengan 'orang-orang kafir' dalam teks ini?
- mengapa nabi nuh tidak mau mengusir orang-orang beriman yang lemah dan miskin?
- bagaimana peran tuhan dalam menilai amal manusia?
- mengapa perbuatan memperdebatkan ayat allah dianggap tidak pantas?
- mengapa allah menantang untuk melakukan tipu daya terhadap-nya?
- mengapa hanya dua pemimpin yang disebut bertakwa dari dua belas orang?

In [23]:
import pandas as pd

PATH_REL = "../data/processed/relevance_pairs_anchor_random.csv"

df = pd.read_csv(PATH_REL)
df.shape


(225342, 6)

In [24]:
df["label"].value_counts(normalize=True)


label
0    0.833333
1    0.166667
Name: proportion, dtype: float64

In [ ]:
pairs_per_query = df.groupby("query_id")["tafsir_id"].count()
pairs_per_query.describe()


count    37557.0
mean         6.0
std          0.0
min          6.0
25%          6.0
50%          6.0
75%          6.0
max          6.0
Name: tafsir_id, dtype: float64

: 